## 19. Нормализация: 2NF

### 🧮 Нормализация: 2NF (Вторая нормальная форма)

**Постановка проблемы:**  
Если в таблице есть составной ключ, и какая-то информация зависит только от части ключа, это нарушает логику: появляются дубликаты, данные сложнее обновлять. Нужно вынести зависимые данные в отдельную таблицу.

**Краткая теория:**  
**2NF** = 1NF + нет частичных зависимостей от составного ключа.

- Все неключевые поля должны зависеть **от всего** ключа, а не от его части
- Часто решается через разбиение таблицы и вынос повторяющихся данных в отдельную сущность

**Пример:**

In [ ]:
# Демонстрация 2NF в SQLite
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Нарушение 2NF: предмет зависит только от student_id
cursor.execute('''CREATE TABLE enrollment_bad (
    student_id INTEGER,
    course_id INTEGER,
    student_name TEXT,
    PRIMARY KEY (student_id, course_id)
)''')
cursor.execute("INSERT INTO enrollment_bad VALUES (1, 101, 'Анна')")
cursor.execute("INSERT INTO enrollment_bad VALUES (1, 102, 'Анна')")
cursor.execute("SELECT * FROM enrollment_bad")
print("🚫 Нарушение 2NF:")
for row in cursor.fetchall():
    print(row)

# Решение: отделяем студентов в отдельную таблицу
cursor.execute("DROP TABLE enrollment_bad")
cursor.execute("CREATE TABLE students (id INTEGER PRIMARY KEY, name TEXT)")
cursor.execute("CREATE TABLE courses (id INTEGER PRIMARY KEY, title TEXT)")
cursor.execute("CREATE TABLE enrollment (
    student_id INTEGER,
    course_id INTEGER,
    PRIMARY KEY (student_id, course_id),
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (course_id) REFERENCES courses(id)
)" )
# Добавляем студентов и курсы
cursor.execute("INSERT INTO students (name) VALUES ('Анна')")
cursor.executemany("INSERT INTO courses (title) VALUES (?)", [("Математика",), ("История",)])
conn.commit()
student_id = cursor.execute("SELECT id FROM students WHERE name = 'Анна'").fetchone()[0]
course_ids = cursor.execute("SELECT id FROM courses").fetchall()
cursor.executemany("INSERT INTO enrollment (student_id, course_id) VALUES (?, ?)",
                   [(student_id, course_id[0]) for course_id in course_ids])
conn.commit()

# Проверка
print("\n✅ После нормализации (2NF):")
cursor.execute('''
SELECT s.name, c.title FROM enrollment e
JOIN students s ON s.id = e.student_id
JOIN courses c ON c.id = e.course_id
''')
for row in cursor.fetchall():
    print(row)